In [13]:
import pandas as pd
import os
import openpyxl
from openpyxl import load_workbook

### 函数

In [14]:
def read_excel(filename,sheet_name,skiprows=None):
    '''读取原始数据, 返回dataframe
    '''
    df = pd.read_excel(filename,sheet_name=sheet_name,skiprows=skiprows)
    return df

def merge_cols(df_left,df_right,how='left'):
    '''
    利用vlookup新增列
    '''
    df = pd.merge(df_left,df_right,how=how)
    return df

def replace_null(df,col_name,replace_default):
    '''
    替换指定列空值
    '''
    df[col_name] = df[col_name].where(df[col_name].notnull(),replace_default)
    return df

def get_ranked_header(filename, sheet_name):
    return pd.read_excel(filename,sheet_name=sheet_name).columns

### main

In [15]:
def main():   
  # 1.读取转换后的当月花费数据
    filename = 'currentdata.xlsx'
    df_append = read_excel(filename,sheet_name='rawdata')
     
  # 2.处理Brand_EN列
    filename = 'AR Monitoring Media Spending.xlsx'
    sheet_name = 'list_Brand_EN'
    col_name = 'Brand_EN'
    replace_default = 'OTHER'
    df_right = read_excel(filename,sheet_name=sheet_name)
    df_append = merge_cols(df_append,df_right,how='left')
    df_append = replace_null(df_append,col_name=col_name,replace_default=replace_default)
    
  # 3. 处理车型_key列
    sheet_name = 'list_车型_key'
    col_name = '车型_key'
    replace_default = 'OTHER'
    df_right = read_excel(filename,sheet_name=sheet_name)
    df_append = merge_cols(df_append,df_right,how='left')
    df_append = replace_null(df_append,col_name=col_name,replace_default=replace_default)
    
  # 4.处理AR_Category列
    sheet_name = 'list_category'
    col_name = 'AR_Category'
    replace_default = 'OTHER'
    df_right = read_excel(filename,sheet_name=sheet_name)
    df_append = merge_cols(df_append,df_right,how='left')
    df_append = replace_null(df_append,col_name=col_name,replace_default=replace_default)
    
  # 5.处理AR_Model列
    sheet_name = 'list_model'
    col_name = 'AR_Model'
    replace_default = 'OTHER'
    df_right = read_excel(filename,sheet_name=sheet_name)
    df_append = merge_cols(df_append,df_right,how='left')
    df_append = replace_null(df_append,col_name=col_name,replace_default=replace_default)
    
  # 6.处理列折算花费(百万)列
    df_append['折算花费(百万)'] = df_append['折算花费（万元）']/100
        
  # 7.remark列空的替换成none
    sheet_name = 'rawdata'
    col_name = 'Remark'
    replace_default = 'None'
    df_append = replace_null(df_append,col_name=col_name,replace_default=replace_default)
    
  # 8.读取AR_Data sheet并与df_append合并且按照原始列排序
    df = read_excel(filename,sheet_name='AR_Data')
    df = df.append(df_append)
    cols = get_ranked_header(filename, sheet_name='AR_Data')
    df = df[cols] # 按原始列排序
    
    return df

In [16]:
df = main()
df.to_excel('result.xlsx',index=None)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [11]:
filename = 'AR Monitoring Media Spending.xlsx'
sheet_name = 'AR_Data'
append_data(filename,sheet_name,df)

KeyboardInterrupt: 

In [14]:
def write_excel(df, writer, sheet_name):
    """
    数据写入到Excel,可以写入不同的sheet
    """
    # 保存新的数据
    book = load_workbook(writer.path)
    writer.book = book
    
    # 清除原来的数据
    idx = book.sheetnames.index('AR_Data')
    book.remove(book.worksheets[idx])
    book.create_sheet('AR_Data', idx)
    
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(excel_writer=writer, sheet_name=sheet_name, index=None)
    writer.close()
    
df = main()
excelPath = r'C:\Users\aliyang4\AlicePython\Excel_pandas_tools\AR Monitoring Media Spending.xlsx'
sheet_name = 'AR_Data'
writer = pd.ExcelWriter(excelPath, engine='openpyxl')
write_excel(df, writer,sheet_name )

KeyError: "['折算花费（百万）'] not found in axis"

### 分步骤

In [2]:
# 读取转换后的当月花费数据
output_filename = 'currentdata.xlsx'
df1 = pd.read_excel(output_filename)
#df1

In [3]:
# 读取《AR Monitoring Media Spending 201401-20200x.xlsx》文件
filename = 'AR Monitoring Media Spending.xlsx'
sheet_name = 'AR_Data'
drop_cols = ['Brand_EN','车型_key','AR_Category','AR_Model','折算花费（百万）']
df2 = pd.read_excel(filename,sheet_name= sheet_name).drop(drop_cols,axis=1)

In [6]:
# df1和df2合并并新增列
df = df2.append(df1)
df_lst_brand = pd.read_excel(filename,sheet_name='list_Brand_EN')
df_lst_model_key = pd.read_excel(filename,sheet_name='list_车型_key')
df_lst_category = pd.read_excel(filename,sheet_name='list_category')
df_lst_model = pd.read_excel(filename,sheet_name='list_model')
df = pd.merge(df,df_lst_brand,how='left')
df = pd.merge(df,df_lst_model_key,how='left')
df = pd.merge(df,df_lst_category,how='left')
df = pd.merge(df,df_lst_model,how='left')
df['Brand_EN'] = df['Brand_EN'].where(df['Brand_EN'].notnull(),'OTHER')
df['车型_key'] = df['车型_key'].where(df['车型_key'].notnull(),'OTHER')
df['AR_Category'] = df['AR_Category'].where(df['AR_Category'].notnull(),'OTHER')
df['AR_Model'] = df['AR_Model'].where(df['AR_Model'].notnull(),'OTHER')
df['折算花费(百万)'] = df['折算花费（万元）']/100

# df.to_excel('result.xlsx',index=None)

In [1]:
import pandas as pd
from append_data import append_data

In [2]:
filename = 'append_data.xlsx'
sheetname = 'Sheet1'
df = pd.DataFrame([[1,2,3],[4,5,6]])
append_data(filename,sheetname,df)